[//]: # ( Fertilizer Calculations with Crop-Specific Nutrient Needs )
[//]: # ( License: MIT License )
[//]: # ( Repository: https://github.com/outobecca/botanical-colabs )

# 🌾 Fertilizer Calculations with Crop-Specific Nutrient Needs
**Version 1.0** | Created: 2025-11-04 | Author: Botanical Colabs Team

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/outobecca/botanical-colabs/blob/main/notebooks/agrology/fertilizer_calculations.ipynb)

## 📋 Overview

**Purpose:** Calculate precise fertilizer requirements based on crop-specific nutrient needs, soil test results, and field characteristics.

**Research Question:** How can we optimize fertilizer application rates to meet crop nutrient demands while minimizing waste and environmental impact?

### 🎯 Use Cases
- Calculate NPK (Nitrogen, Phosphorus, Potassium) requirements for different crops
- Account for existing soil nutrients from soil tests
- Determine fertilizer product quantities needed
- Compare different fertilizer products and formulations
- Generate application recommendations by crop type
- Estimate costs and optimize fertilizer selection

### 🌱 Supported Crops

| Category | Crops | Nutrient Profile |
|----------|-------|------------------|
| **Vegetables** | Tomato, Lettuce, Pepper, Cucumber | High N, Moderate P-K |
| **Grains** | Corn, Wheat, Barley | Moderate-High N |
| **Legumes** | Soybean, Peas, Beans | Low N, Moderate P-K |
| **Root Crops** | Potato, Carrot, Beet | Moderate N, High K |

### ⚠️ Notes
- Values are general guidelines - adjust for local conditions
- Always perform soil tests for accurate recommendations
- Consider organic matter and previous crop history
- Follow local regulations for fertilizer application


## 📚 Background & Methodology

### Scientific Context

Proper fertilizer management requires understanding:
- **Crop Nutrient Demands:** Different crops have varying NPK requirements at different growth stages
- **Soil Nutrient Supply:** Existing soil nutrients reduce fertilizer needs
- **Fertilizer Efficiency:** Not all applied nutrients are available to plants
- **Environmental Stewardship:** Precision application prevents nutrient runoff and pollution

### Nutrient Requirements

**Macronutrients (NPK):**
- **Nitrogen (N):** Promotes vegetative growth, leaf development, and protein synthesis
- **Phosphorus (P):** Essential for root development, flowering, and energy transfer
- **Potassium (K):** Improves disease resistance, water regulation, and fruit quality

### Methodology
1. **Crop Selection** - Choose crop and target yield
2. **Soil Testing** - Input existing soil nutrient levels
3. **Nutrient Budget** - Calculate crop needs minus soil supply
4. **Fertilizer Selection** - Choose appropriate product(s)
5. **Rate Calculation** - Determine application quantities
6. **Recommendations** - Generate application schedule

### Expected Outputs
- Nutrient requirement calculations by crop
- Fertilizer product recommendations
- Application rates (kg/ha or lb/acre)
- Cost estimates
- Comparative analysis of fertilizer options


## ⚙️ Step 1: Installation and Configuration

Run the cells below to install libraries and configure your analysis.


In [ ]:
# ============================================================================
# Library Installation and Import
# ============================================================================
"""
Installs required Python libraries.
Run this cell first.
"""

# Installation
!pip install -q pandas numpy matplotlib seaborn ipywidgets

# Core imports
from typing import Dict, Optional, List, Tuple, Any
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries installed successfully")


In [ ]:
# ============================================================================
# Interactive Configuration
# ============================================================================

# Crop selection (FORM)
print("🌾 SELECT CROP TYPE:")
crop_options = {
    '1': 'Tomato',
    '2': 'Lettuce',
    '3': 'Corn (Maize)',
    '4': 'Wheat',
    '5': 'Potato',
    '6': 'Soybean',
    '7': 'Pepper',
    '8': 'Cucumber',
    '9': 'Carrot',
    '10': 'Custom (enter values manually)'
}

for key, crop in crop_options.items():
    print(f"  [{key}] {crop}")

CROP_CHOICE = input("Enter choice (1-10): ").strip() or '1'
SELECTED_CROP = crop_options.get(CROP_CHOICE, 'Tomato')

# Field parameters (FORM)
print("\n📏 FIELD PARAMETERS:")
FIELD_AREA_HA = float(input("Field area (hectares, default 1.0): ").strip() or '1.0')
TARGET_YIELD = float(input(f"Target yield for {SELECTED_CROP} (tonnes/ha, default varies by crop): ").strip() or '0')

# Soil test results (FORM)
print("\n🧪 SOIL TEST RESULTS (current nutrient levels):")
print("   (Leave blank to use default medium fertility values)")
SOIL_N_KG = float(input("Soil Nitrogen (kg/ha, default 40): ").strip() or '40')
SOIL_P_KG = float(input("Soil Phosphorus (kg/ha, default 25): ").strip() or '25')
SOIL_K_KG = float(input("Soil Potassium (kg/ha, default 150): ").strip() or '150')

# Unit preference (FORM)
print("\n📐 UNIT PREFERENCE:")
print("  [1] Metric (kg/ha)")
print("  [2] Imperial (lb/acre)")
UNIT_CHOICE = input("Enter choice (1-2, default 1): ").strip() or '1'
USE_METRIC = UNIT_CHOICE == '1'

print("\n✅ Configuration complete!")
print(f"   Crop: {SELECTED_CROP}")
print(f"   Field area: {FIELD_AREA_HA} ha")
print(f"   Soil N: {SOIL_N_KG} kg/ha, P: {SOIL_P_KG} kg/ha, K: {SOIL_K_KG} kg/ha")
print(f"   Units: {'Metric (kg/ha)' if USE_METRIC else 'Imperial (lb/acre)'}")


## 🔧 Step 2: Crop Nutrient Database and Helper Functions

Database of crop-specific nutrient requirements and calculation utilities.


In [ ]:
# ============================================================================
# Crop Nutrient Requirements Database
# ============================================================================
"""
Nutrient requirements in kg/ha for different crops.
Values are general guidelines based on agricultural research.
Adjust based on local conditions and soil tests.
"""

CROP_NUTRIENT_DATABASE = {
    'Tomato': {
        'N': 150,  # kg/ha
        'P': 80,   # kg/ha P2O5 equivalent
        'K': 200,  # kg/ha K2O equivalent
        'default_yield': 50,  # tonnes/ha
        'category': 'Vegetable',
        'description': 'High nutrient demands, especially K for fruit quality'
    },
    'Lettuce': {
        'N': 100,
        'P': 50,
        'K': 120,
        'default_yield': 30,
        'category': 'Leafy Vegetable',
        'description': 'High N for leaf growth, moderate P-K'
    },
    'Corn (Maize)': {
        'N': 180,
        'P': 70,
        'K': 80,
        'default_yield': 10,
        'category': 'Grain',
        'description': 'High N demands, moderate P-K'
    },
    'Wheat': {
        'N': 120,
        'P': 60,
        'K': 50,
        'default_yield': 6,
        'category': 'Grain',
        'description': 'Moderate nutrient requirements'
    },
    'Potato': {
        'N': 120,
        'P': 70,
        'K': 200,
        'default_yield': 40,
        'category': 'Root Crop',
        'description': 'High K for tuber development'
    },
    'Soybean': {
        'N': 40,   # Low due to N-fixation
        'P': 60,
        'K': 100,
        'default_yield': 3,
        'category': 'Legume',
        'description': 'Low N (fixes own), moderate P-K'
    },
    'Pepper': {
        'N': 140,
        'P': 75,
        'K': 180,
        'default_yield': 25,
        'category': 'Vegetable',
        'description': 'High nutrient demands, similar to tomato'
    },
    'Cucumber': {
        'N': 130,
        'P': 65,
        'K': 150,
        'default_yield': 35,
        'category': 'Vegetable',
        'description': 'Moderate-high nutrient requirements'
    },
    'Carrot': {
        'N': 80,
        'P': 60,
        'K': 160,
        'default_yield': 45,
        'category': 'Root Crop',
        'description': 'Moderate N, high K for root development'
    }
}

# ============================================================================
# Fertilizer Products Database
# ============================================================================

FERTILIZER_PRODUCTS = {
    'Urea': {'N': 46, 'P': 0, 'K': 0, 'cost_per_kg': 0.50},
    'Ammonium Nitrate': {'N': 34, 'P': 0, 'K': 0, 'cost_per_kg': 0.45},
    'DAP (Diammonium Phosphate)': {'N': 18, 'P': 46, 'K': 0, 'cost_per_kg': 0.60},
    'TSP (Triple Superphosphate)': {'N': 0, 'P': 46, 'K': 0, 'cost_per_kg': 0.55},
    'Muriate of Potash (KCl)': {'N': 0, 'P': 0, 'K': 60, 'cost_per_kg': 0.40},
    'NPK 15-15-15': {'N': 15, 'P': 15, 'K': 15, 'cost_per_kg': 0.55},
    'NPK 20-10-10': {'N': 20, 'P': 10, 'K': 10, 'cost_per_kg': 0.52},
    'NPK 10-20-20': {'N': 10, 'P': 20, 'K': 20, 'cost_per_kg': 0.58}
}

# ============================================================================
# Helper Functions
# ============================================================================

def get_crop_requirements(crop_name: str, target_yield: Optional[float] = None) -> Dict[str, Any]:
    """
    Get nutrient requirements for a crop.
    Adjusts based on target yield if provided.
    """
    if crop_name not in CROP_NUTRIENT_DATABASE:
        raise ValueError(f"Crop '{crop_name}' not found in database")
    
    crop_data = CROP_NUTRIENT_DATABASE[crop_name].copy()
    
    # Adjust for target yield
    if target_yield and target_yield > 0:
        yield_factor = target_yield / crop_data['default_yield']
        crop_data['N'] *= yield_factor
        crop_data['P'] *= yield_factor
        crop_data['K'] *= yield_factor
        crop_data['adjusted_yield'] = target_yield
    
    return crop_data

def calculate_fertilizer_need(crop_requirement: float, soil_level: float, 
                             efficiency: float = 0.7) -> float:
    """
    Calculate fertilizer need accounting for soil supply and efficiency.
    
    Args:
        crop_requirement: Crop nutrient need (kg/ha)
        soil_level: Current soil nutrient level (kg/ha)
        efficiency: Fertilizer efficiency factor (0-1, default 0.7)
    
    Returns:
        Fertilizer needed (kg/ha)
    """
    net_need = max(0, crop_requirement - soil_level)
    fertilizer_need = net_need / efficiency
    return fertilizer_need

def calculate_product_amount(nutrient_need: float, product_concentration: float) -> float:
    """
    Calculate fertilizer product amount needed.
    
    Args:
        nutrient_need: Nutrient needed (kg/ha)
        product_concentration: % of nutrient in product
    
    Returns:
        Product amount needed (kg/ha)
    """
    if product_concentration == 0:
        return 0
    return (nutrient_need / product_concentration) * 100

def kg_per_ha_to_lb_per_acre(kg_ha: float) -> float:
    """Convert kg/ha to lb/acre."""
    return kg_ha * 0.893

def format_value(value: float, use_metric: bool = True) -> str:
    """Format value with appropriate units."""
    if use_metric:
        return f"{value:.1f} kg/ha"
    else:
        return f"{kg_per_ha_to_lb_per_acre(value):.1f} lb/acre"

print("✅ Crop database and helper functions loaded")
print(f"   {len(CROP_NUTRIENT_DATABASE)} crops in database")
print(f"   {len(FERTILIZER_PRODUCTS)} fertilizer products available")


## 🧮 Step 3: Calculate Nutrient Requirements

Calculate crop-specific nutrient needs and account for soil supply.


In [ ]:
# ============================================================================
# Nutrient Requirement Calculations
# ============================================================================

# Get crop requirements
if SELECTED_CROP == 'Custom (enter values manually)':
    print("\n🌾 CUSTOM CROP - Enter nutrient requirements:")
    crop_n = float(input("Nitrogen requirement (kg/ha): ").strip() or '100')
    crop_p = float(input("Phosphorus requirement (kg/ha): ").strip() or '50')
    crop_k = float(input("Potassium requirement (kg/ha): ").strip() or '100')
    
    crop_data = {
        'N': crop_n,
        'P': crop_p,
        'K': crop_k,
        'category': 'Custom',
        'description': 'User-defined values'
    }
else:
    crop_data = get_crop_requirements(
        SELECTED_CROP, 
        TARGET_YIELD if TARGET_YIELD > 0 else None
    )

# Calculate fertilizer needs
FERTILIZER_N = calculate_fertilizer_need(crop_data['N'], SOIL_N_KG)
FERTILIZER_P = calculate_fertilizer_need(crop_data['P'], SOIL_P_KG)
FERTILIZER_K = calculate_fertilizer_need(crop_data['K'], SOIL_K_KG)

# Display results
print("\n" + "="*60)
print(f"📊 NUTRIENT BUDGET for {SELECTED_CROP}")
print("="*60)

if 'category' in crop_data:
    print(f"\nCrop Category: {crop_data['category']}")
    print(f"Description: {crop_data['description']}")

print("\n--- Nitrogen (N) ---")
print(f"Crop requirement: {format_value(crop_data['N'], USE_METRIC)}")
print(f"Soil supply:      {format_value(SOIL_N_KG, USE_METRIC)}")
print(f"Fertilizer need:  {format_value(FERTILIZER_N, USE_METRIC)}")

print("\n--- Phosphorus (P) ---")
print(f"Crop requirement: {format_value(crop_data['P'], USE_METRIC)}")
print(f"Soil supply:      {format_value(SOIL_P_KG, USE_METRIC)}")
print(f"Fertilizer need:  {format_value(FERTILIZER_P, USE_METRIC)}")

print("\n--- Potassium (K) ---")
print(f"Crop requirement: {format_value(crop_data['K'], USE_METRIC)}")
print(f"Soil supply:      {format_value(SOIL_K_KG, USE_METRIC)}")
print(f"Fertilizer need:  {format_value(FERTILIZER_K, USE_METRIC)}")

print("\n" + "="*60)
print(f"Note: Calculations assume 70% fertilizer efficiency")
print("="*60)


## 🧪 Step 4: Fertilizer Product Recommendations

Calculate required amounts of different fertilizer products.


In [ ]:
# ============================================================================
# Fertilizer Product Calculations
# ============================================================================

# Calculate amounts for each product
fertilizer_recommendations = {}

for product_name, product_data in FERTILIZER_PRODUCTS.items():
    # Calculate amount needed for each nutrient
    n_amount = calculate_product_amount(FERTILIZER_N, product_data['N']) if FERTILIZER_N > 0 else 0
    p_amount = calculate_product_amount(FERTILIZER_P, product_data['P']) if FERTILIZER_P > 0 else 0
    k_amount = calculate_product_amount(FERTILIZER_K, product_data['K']) if FERTILIZER_K > 0 else 0
    
    # For single-nutrient products, use the appropriate amount
    if product_data['N'] > 0 and product_data['P'] == 0 and product_data['K'] == 0:
        total_amount = n_amount
        provides = 'N'
    elif product_data['P'] > 0 and product_data['N'] == 0 and product_data['K'] == 0:
        total_amount = p_amount
        provides = 'P'
    elif product_data['K'] > 0 and product_data['N'] == 0 and product_data['P'] == 0:
        total_amount = k_amount
        provides = 'K'
    else:
        # For multi-nutrient products, use maximum
        total_amount = max(n_amount, p_amount, k_amount)
        provides = 'NPK'
    
    if total_amount > 0:
        total_field_amount = total_amount * FIELD_AREA_HA
        total_cost = total_field_amount * product_data['cost_per_kg']
        
        fertilizer_recommendations[product_name] = {
            'amount_per_ha': total_amount,
            'total_amount': total_field_amount,
            'cost': total_cost,
            'provides': provides,
            'n_content': product_data['N'],
            'p_content': product_data['P'],
            'k_content': product_data['K']
        }

# Display recommendations
print("\n" + "="*80)
print("🧪 FERTILIZER PRODUCT RECOMMENDATIONS")
print("="*80)

print(f"\nField area: {FIELD_AREA_HA} hectares")
print("\nSingle-Nutrient Products (apply as needed for each nutrient):")
print("-" * 80)

# Sort by provides type
for product_name, data in sorted(fertilizer_recommendations.items(), 
                                  key=lambda x: (x[1]['provides'], x[1]['cost'])):
    if data['provides'] in ['N', 'P', 'K']:
        unit = "kg/ha" if USE_METRIC else "lb/acre"
        amount = data['amount_per_ha'] if USE_METRIC else kg_per_ha_to_lb_per_acre(data['amount_per_ha'])
        
        print(f"\n{product_name} ({data['n_content']}-{data['p_content']}-{data['k_content']}):")
        print(f"  Application rate: {amount:.1f} {unit}")
        print(f"  Total needed: {data['total_amount']:.1f} kg")
        print(f"  Estimated cost: ${data['cost']:.2f}")
        print(f"  Provides: {data['provides']}")

print("\n" + "-" * 80)
print("\nComplete NPK Products (single application):")
print("-" * 80)

for product_name, data in sorted(fertilizer_recommendations.items(), 
                                  key=lambda x: x[1]['cost']):
    if data['provides'] == 'NPK':
        unit = "kg/ha" if USE_METRIC else "lb/acre"
        amount = data['amount_per_ha'] if USE_METRIC else kg_per_ha_to_lb_per_acre(data['amount_per_ha'])
        
        print(f"\n{product_name} ({data['n_content']}-{data['p_content']}-{data['k_content']}):")
        print(f"  Application rate: {amount:.1f} {unit}")
        print(f"  Total needed: {data['total_amount']:.1f} kg")
        print(f"  Estimated cost: ${data['cost']:.2f}")

print("\n" + "="*80)
print("⚠️  Note: Costs are estimates. Check with local suppliers for current prices.")
print("    Multi-nutrient products may require supplementation with single-nutrient fertilizers.")
print("="*80)


## 📊 Step 5: Visualizations

Create visual comparisons of nutrient requirements and fertilizer options.


In [ ]:
# ============================================================================
# Visualizations
# ============================================================================

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle(f'Fertilizer Analysis for {SELECTED_CROP}', fontsize=16, fontweight='bold')

# 1. Nutrient Budget Bar Chart
ax1 = axes[0, 0]
nutrients = ['Nitrogen', 'Phosphorus', 'Potassium']
crop_needs = [crop_data['N'], crop_data['P'], crop_data['K']]
soil_supply = [SOIL_N_KG, SOIL_P_KG, SOIL_K_KG]
fertilizer_needs = [FERTILIZER_N, FERTILIZER_P, FERTILIZER_K]

x = np.arange(len(nutrients))
width = 0.25

ax1.bar(x - width, crop_needs, width, label='Crop Requirement', color='#2ecc71')
ax1.bar(x, soil_supply, width, label='Soil Supply', color='#8b4513')
ax1.bar(x + width, fertilizer_needs, width, label='Fertilizer Need', color='#e74c3c')

ax1.set_ylabel('Amount (kg/ha)')
ax1.set_title('Nutrient Budget Comparison')
ax1.set_xticks(x)
ax1.set_xticklabels(nutrients)
ax1.legend()
ax1.grid(axis='y', alpha=0.3)

# 2. Fertilizer Cost Comparison
ax2 = axes[0, 1]
if fertilizer_recommendations:
    products = list(fertilizer_recommendations.keys())[:6]  # Top 6
    costs = [fertilizer_recommendations[p]['cost'] for p in products]
    colors = ['#3498db' if fertilizer_recommendations[p]['provides'] == 'NPK' else '#95a5a6' 
              for p in products]
    
    ax2.barh(products, costs, color=colors)
    ax2.set_xlabel('Estimated Cost ($)')
    ax2.set_title('Fertilizer Product Cost Comparison')
    ax2.grid(axis='x', alpha=0.3)
else:
    ax2.text(0.5, 0.5, 'No fertilizer needed', ha='center', va='center')
    ax2.set_title('Fertilizer Product Cost Comparison')

# 3. NPK Ratio Pie Chart
ax3 = axes[1, 0]
npk_needs = [FERTILIZER_N, FERTILIZER_P, FERTILIZER_K]
if sum(npk_needs) > 0:
    colors_pie = ['#e74c3c', '#f39c12', '#9b59b6']
    ax3.pie(npk_needs, labels=['N', 'P', 'K'], autopct='%1.1f%%', 
            colors=colors_pie, startangle=90)
    ax3.set_title('Fertilizer NPK Distribution')
else:
    ax3.text(0.5, 0.5, 'Soil sufficient', ha='center', va='center')
    ax3.set_title('Fertilizer NPK Distribution')

# 4. Crop Comparison Table
ax4 = axes[1, 1]
ax4.axis('tight')
ax4.axis('off')

# Create comparison with other crops
comparison_crops = list(CROP_NUTRIENT_DATABASE.keys())[:5]
table_data = []
for crop in comparison_crops:
    cdata = CROP_NUTRIENT_DATABASE[crop]
    highlight = '→ ' if crop == SELECTED_CROP else '   '
    table_data.append([
        highlight + crop,
        f"{cdata['N']:.0f}",
        f"{cdata['P']:.0f}",
        f"{cdata['K']:.0f}"
    ])

table = ax4.table(cellText=table_data,
                  colLabels=['Crop', 'N', 'P', 'K'],
                  cellLoc='left',
                  loc='center',
                  colWidths=[0.4, 0.2, 0.2, 0.2])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1, 2)

# Style header
for i in range(4):
    table[(0, i)].set_facecolor('#34495e')
    table[(0, i)].set_text_props(weight='bold', color='white')

ax4.set_title('Crop Nutrient Requirements (kg/ha)', pad=20)

plt.tight_layout()
plt.show()

print("\n✅ Visualizations generated successfully")


## 📝 Step 6: Application Recommendations

Generate practical application guidelines.


In [ ]:
# ============================================================================
# Application Recommendations
# ============================================================================

print("\n" + "="*80)
print("📝 APPLICATION RECOMMENDATIONS")
print("="*80)

print(f"\nCrop: {SELECTED_CROP}")
print(f"Field Area: {FIELD_AREA_HA} ha")
if TARGET_YIELD > 0:
    print(f"Target Yield: {TARGET_YIELD} tonnes/ha")

print("\n--- Recommended Application Strategy ---\n")

# Base recommendation
if FERTILIZER_N > 0 or FERTILIZER_P > 0 or FERTILIZER_K > 0:
    print("OPTION 1: Single-Nutrient Approach (Most Precise)")
    print("Apply separate fertilizers for each nutrient:\n")
    
    if FERTILIZER_N > 0:
        urea_amount = calculate_product_amount(FERTILIZER_N, FERTILIZER_PRODUCTS['Urea']['N'])
        print(f"  • Nitrogen: {urea_amount * FIELD_AREA_HA:.1f} kg Urea (46-0-0)")
        print(f"    - Apply in 2-3 split applications during growing season")
    
    if FERTILIZER_P > 0:
        dap_amount = calculate_product_amount(FERTILIZER_P, FERTILIZER_PRODUCTS['DAP (Diammonium Phosphate)']['P'])
        print(f"  • Phosphorus: {dap_amount * FIELD_AREA_HA:.1f} kg DAP (18-46-0)")
        print(f"    - Apply at planting or pre-plant")
    
    if FERTILIZER_K > 0:
        kcl_amount = calculate_product_amount(FERTILIZER_K, FERTILIZER_PRODUCTS['Muriate of Potash (KCl)']['K'])
        print(f"  • Potassium: {kcl_amount * FIELD_AREA_HA:.1f} kg Muriate of Potash (0-0-60)")
        print(f"    - Apply at planting or pre-plant")
    
    print("\n" + "-"*80)
    print("\nOPTION 2: Complete NPK Fertilizer (More Convenient)")
    print("Apply a balanced fertilizer product:\n")
    
    # Find best NPK match
    npk_products = {k: v for k, v in fertilizer_recommendations.items() 
                    if v['provides'] == 'NPK'}
    if npk_products:
        best_npk = min(npk_products.items(), key=lambda x: x[1]['cost'])
        product_name, product_data = best_npk
        print(f"  • {product_name}: {product_data['total_amount']:.1f} kg total")
        print(f"    - Cost: ${product_data['cost']:.2f}")
        print(f"    - Apply at planting with possible supplementation later")
        print(f"    - Note: May need additional single-nutrient fertilizer to meet all requirements")
    
    print("\n" + "-"*80)
    print("\n--- Timing and Method ---\n")
    print("1. Pre-plant: Apply P and K fertilizers and incorporate into soil")
    print("2. At planting: Apply starter fertilizer if using complete NPK")
    print("3. Side-dress: Apply nitrogen in 2-3 split applications during growth")
    print("4. Foliar: Consider foliar feeding for micronutrients (not calculated here)")
    
    print("\n--- Best Practices ---\n")
    print("• Conduct soil tests annually to adjust recommendations")
    print("• Split nitrogen applications to reduce leaching losses")
    print("• Apply when soil moisture is adequate for incorporation")
    print("• Avoid application before heavy rain to prevent runoff")
    print("• Calibrate equipment for accurate application rates")
    print("• Keep records of applications for future reference")
    print("• Consider slow-release fertilizers in sandy soils")
    
else:
    print("\n✅ Soil nutrient levels are SUFFICIENT for this crop!")
    print("No fertilizer application needed based on current soil test results.\n")
    print("Recommendations:")
    print("• Monitor plant growth and nutrient status")
    print("• Re-test soil mid-season or for next crop")
    print("• Maintain organic matter through crop residue or compost")

print("\n" + "="*80)
print("⚠️  IMPORTANT DISCLAIMERS")
print("="*80)
print("""
• These are general recommendations based on typical crop requirements
• Local soil conditions, climate, and management practices vary
• Always perform soil tests before applying fertilizers
• Consult with local agricultural extension services
• Follow local regulations for fertilizer application
• Consider environmental impact and water quality
• Calculations assume 70% fertilizer efficiency - adjust for local conditions
""")
print("="*80)


## 💾 Step 7: Export Results

Save calculations and recommendations for future reference.


In [ ]:
# ============================================================================
# Export Results
# ============================================================================

# Create summary dataframe
summary_data = {
    'Crop': [SELECTED_CROP],
    'Field_Area_ha': [FIELD_AREA_HA],
    'Crop_N_Requirement_kg_ha': [crop_data['N']],
    'Crop_P_Requirement_kg_ha': [crop_data['P']],
    'Crop_K_Requirement_kg_ha': [crop_data['K']],
    'Soil_N_kg_ha': [SOIL_N_KG],
    'Soil_P_kg_ha': [SOIL_P_KG],
    'Soil_K_kg_ha': [SOIL_K_KG],
    'Fertilizer_N_Needed_kg_ha': [FERTILIZER_N],
    'Fertilizer_P_Needed_kg_ha': [FERTILIZER_P],
    'Fertilizer_K_Needed_kg_ha': [FERTILIZER_K],
    'Total_N_Field_kg': [FERTILIZER_N * FIELD_AREA_HA],
    'Total_P_Field_kg': [FERTILIZER_P * FIELD_AREA_HA],
    'Total_K_Field_kg': [FERTILIZER_K * FIELD_AREA_HA],
    'Date': [datetime.now().strftime('%Y-%m-%d')]
}

summary_df = pd.DataFrame(summary_data)

# Create fertilizer products dataframe
if fertilizer_recommendations:
    products_data = []
    for product, data in fertilizer_recommendations.items():
        products_data.append({
            'Product': product,
            'NPK_Ratio': f"{data['n_content']}-{data['p_content']}-{data['k_content']}",
            'Application_Rate_kg_ha': data['amount_per_ha'],
            'Total_Amount_kg': data['total_amount'],
            'Estimated_Cost_USD': data['cost'],
            'Provides': data['provides']
        })
    products_df = pd.DataFrame(products_data)
else:
    products_df = pd.DataFrame()

# Display summary
print("\n" + "="*80)
print("📋 CALCULATION SUMMARY")
print("="*80)
print("\nNutrient Requirements Summary:")
display(summary_df.T)

if not products_df.empty:
    print("\nFertilizer Products Summary:")
    display(products_df)

# Save to CSV
try:
    filename_base = f"fertilizer_calc_{SELECTED_CROP.replace(' ', '_')}_{datetime.now().strftime('%Y%m%d')}"
    summary_df.to_csv(f"{filename_base}_summary.csv", index=False)
    if not products_df.empty:
        products_df.to_csv(f"{filename_base}_products.csv", index=False)
    print(f"\n✅ Results saved to:")
    print(f"   - {filename_base}_summary.csv")
    if not products_df.empty:
        print(f"   - {filename_base}_products.csv")
except Exception as e:
    print(f"\n⚠️  Could not save files: {e}")
    print("   (This is normal in Colab - files are available in the session)")

print("\n" + "="*80)


## 📚 Citations and Data Sources

### Nutrient Requirement Data
- Crop nutrient requirements based on:
  - FAO Fertilizer and Plant Nutrition Guide
  - University Extension Service recommendations
  - International Plant Nutrition Institute (IPNI) guidelines

### Methodology References
- Fertilizer efficiency factors from agricultural research literature
- NPK conversion factors follow standard agricultural chemistry
- Soil test interpretation from regional extension guidelines

### Important Notes
- All values are general guidelines and may vary by:
  - Local soil conditions
  - Climate and growing season length
  - Crop variety and management practices
  - Previous crop history and residual nutrients

### Recommended Reading
- "Nutrient Use in Crop Production" - Zublena et al.
- "Soil Fertility and Fertilizers" - Havlin et al.
- FAO Soils Portal: http://www.fao.org/soils-portal

### Disclaimer
This tool provides educational estimates. Always consult with:
- Certified crop advisors
- Agricultural extension services
- Soil testing laboratories
- Local regulatory agencies

---

**License:** MIT License  
**Repository:** https://github.com/outobecca/botanical-colabs  
**Last Updated:** 2025-11-04
